# Dependencies and Definitions

In [4]:
# Install the python-Levenshtein package using pip
#!pip install python-Levenshtein

# Import necessary libraries and modules

import Levenshtein  # Library for computing Levenshtein (edit) distance

# import numpy and pandas using standard names
import numpy as np  
import pandas as pd

import re  # Regular expressions module for string operations
from tqdm import tqdm  # Module for displaying progress bars

# Below we define some functions we will use later.

def find_domain_name(x):
    """
    Extracts the domain name from an email address.

    Args:
        x (str): The input email address.

    Returns:
        str: The domain name part of the email address.
    """
    # If '@.' return all whole string since this is likely a typo we need to check further.
    if "@." in x:
        return x
    try:
        idx = x.index('@')  # Find the index of the '@' symbol
        return x[idx+1:]  # Return the substring after the '@' symbol
    except:
        return x  # If no '@' symbol is found, return the original input

def split_and_join(x):
    """
    Removes spaces from a string.

    Args:
        x (str): The input string.

    Returns:
        str: The input string with spaces removed.
    """
    return ''.join(x.split())  # Split the string into words, join them without spaces

def remove_punctuation(input_string):
    """
    Removes punctuation from a given string.

    Args:
        input_string (str): The input string containing punctuation.

    Returns:
        str: The input string with punctuation removed.
    """
    # Define a regular expression pattern to match punctuation
    punctuation_pattern = r'[^\w\s]'
    
    # Use re.sub() to replace punctuation with an empty string
    clean_string = re.sub(punctuation_pattern, '', input_string)
    
    return clean_string  # Return the modified string


# Questions

In [5]:
df = pd.read_csv('Value_of_Energy_Cost_Savings_Program_Savings_for_Businesses.csv')

## Question 1

Here we will answer the question: How many different companies are represented in the data set? To do this, we need to clean the data since many companies are repeated or misspelled.

In [6]:
# First we lowercase the company name and apply the function split_and_join. 
# split_and_join removes whitespace and concatenates the string.
df['Company Name'] = df['Company Name'].str.lower().apply(split_and_join)

# We notice on visual inspection that many company names are duplicated, but the 
# duplicated versions have many duplicated. apostrophes. The code below uses the 're' module 
# to remove them. That is it replaces the string '''''' with '.
df['Company Name'] = df['Company Name'].astype(str).apply(
    lambda x:re.sub("'+","'",x))

# Next, we remove any punctuation from the company name.
df['Company Name'] = df['Company Name'].astype(str).apply(remove_punctuation)

# Finally, we remove a common misspelling of "corporation".
df['Company Name'] = df['Company Name'].astype(str).apply(lambda x: x.replace(
    'coporation','corporation'))

# The list companies contains the resulting unique set of company names.
companies = df['Company Name'].unique()

# Next, we test whether the company names are unique by computing the Levenshtein word distance between
# the remaining companies. The Levenshtein distance measures how many edits it would take to make two words identical.
length = len(companies)

# We initialize the "distance matrix" so that each element is infinite. This is done out of convenience since we 
# want to ignore redudant and diagonal matrix elements.
dist_matrix = np.full((length,length),float('inf'))

# Finally, we measure the word distance between all unique pairs of words.
# We restrict to i<j since the word distance metric is symmetric, i.e. the distance between word[i] and word[j]
# is the same as the distance between word[j] and word[i]
for i in tqdm(range(length-1)):
    for j in range(i+1,length):
        dist_matrix[i,j] = Levenshtein.distance(companies[i],
                                                companies[j])

print(f'Total number of unique companies so far: {len(companies)}')

100%|███████████████████████████████████████| 761/761 [00:00<00:00, 3595.93it/s]

Total number of unique companies so far: 762


In [7]:
# Next, let's look at all company names which have a Levenshtein distance <= 2.

for i in range(length-1):
    for j in range(i+1,length):
        if dist_matrix[i,j]<=2:
            print(f"Company {i}: {companies[i]}")
            print(f"Company {j}: {companies[j]}")
            print('-'*50)

Company 162: dlxindustriesinc
Company 700: dnjindustriesinc
--------------------------------------------------
Company 270: jetrocashcarryenterprisesinc
Company 647: jetrocashcarryenterprisesllc
--------------------------------------------------
Company 329: mcndesigninc
Company 339: mldesigninc
--------------------------------------------------
Company 494: tourneauinc
Company 757: tourneaullc
--------------------------------------------------
Company 551: 120wallstreetllc
Company 557: 40wallstreetllc
--------------------------------------------------
Company 581: kickstarterinc
Company 649: kickstarterpbc
--------------------------------------------------


We notice that the pairs (270,647), (494, 757) and (581,649) are very similar, the only difference is whether the company name inolves "INC" or "LLC" or "INC" vs "PBC". A priori, it is not clear to me whether these companies should be considered the same or different. However, given that "INC" and "LLC" are legally two different terms, I've opted to consider them to be separate companies here. With this assumption, the total number of companies is: len(companies) = 762.

## Question 2

What is the total number of jobs created for businesses in Queens?

To answer this question we will restrict our dataframe df to rows where the borough is Queens, index into the 'Job created' column, and then sum all the entries. This is done in the code below and the final answer is 1196.

In [8]:
num_queens_jobs = df[df['Borough']=='QUEENS']['Job created'].sum()

print(f'Total number of jobs created in Queens is: {int(num_queens_jobs)}.')

Total number of jobs created in Queens is: 1196.


## Question 3

How many different unique email domains names are there in the data set?

To answer this, we need to extract the contents of the email string after the @ symbol, if it is present. If the @ symbol is not present we will assume the url given is meant to be the domain name. An exception is if the string contains "@." which we assume is a typo. Specifically, the email address 'reinfsupply@.net' in df['company email'] we suspect is a typo.

In [10]:
# First, we'll standardize the email addresses by lowercasing and removing any potential white space.
df['company email'] = df['company email'].str.lower().astype(str).apply(split_and_join)

#df['company email'] = df['company email'].str.replace('www.','')
df['domains'] = df['company email'].apply(find_domain_name)
domains = df['domains'].unique()
print(f'Number of unique domains found so far: {len(domains)}.')

Number of unique domains found so far: 601.


As in our solution to question 1, we will use the Levenshtein distance metric to look for common typos in the email address domain names.

In [11]:
domain_length = len(domains)

# We again initialize the "distance matrix" so that each element is infinite.
dist_matrix_domain = np.full((length,length),float('inf'))

# Finally, we measure the word distance between all unique pairs of domains.
# We again restrict to i<j since the word distance metric is symmetric.

for i in tqdm(range(domain_length-1)):
    for j in range(i+1,domain_length):
        dist_matrix_domain[i,j] = Levenshtein.distance(domains[i],
                                                domains[j])

100%|███████████████████████████████████████| 600/600 [00:00<00:00, 4630.95it/s]


With emails its more difficult to identify whether something is a typo or not. After all, it is not uncommon for emails to differ by just one letter. For example "NYU.edu" and "YU.edu" correspond to different institutions, New York University and Yeshiva University. Nevertheless, its still important to check for potential obvious typos in the email addresses.

To start, we look for potentially redundant emails by computing the Levenshtein distance between two domain names. Then for each potentially redundant pair, we check to see if the company name agree. If they agree, we add the pair of domain names to the set "domains_redundant".

In [12]:
# First we collect all domain names which have a Levenshtein distance <= 5.

domain_pairs = []

for i in range(length-1):
    for j in range(i+1,length):
        if dist_matrix_domain[i,j]<=5:
            domain_pairs.append((domains[i],domains[j]))

In [13]:
# Next, for each pair of domain names, we check if the domain names correspond to the same company by 
# checking whether the company name, phone number, and contact name all agree.

import itertools
domains_check = set()

for domain_pair in domain_pairs:
    
    domain_1, domain_2 = domain_pair
    
    company_1 = df[(df['domains']==domain_1)]
    company_2 = df[(df['domains']==domain_2)]
    
    for i,j in itertools.product(company_1.index,company_2.index):
        
        if (company_1['Company Name'][i]==company_2['Company Name'][j]):

                print(f"Company {i}: {company_1['Company Name'][i]}. Email: {company_1['company email'][i]}")
                print(f"Company {j}: {company_2['Company Name'][j]}. Email: {company_2['company email'][j]}")
                print('-'*80)
                
                domains_check.add((domain_1,domain_2))
                break

Company 1787: kingsland359llc. Email: ginabws@gmail.com
Company 299: kingsland359llc. Email: ebond203@aol.com
--------------------------------------------------------------------------------
Company 286: josiahmcelhenystudio. Email: markshortliffe@gmail.com
Company 1777: josiahmcelhenystudio. Email: josiahmcstudio@yahoo.com
--------------------------------------------------------------------------------
Company 28: aidsvaccineadvocacycoalition. Email: marie@avac.com
Company 605: aidsvaccineadvocacycoalition. Email: marie@avac.org
--------------------------------------------------------------------------------
Company 72: belmontmetals. Email: jbehrendt@belomtmetals.com
Company 1623: belmontmetals. Email: jbehrendt@belmontmetals.com
--------------------------------------------------------------------------------
Company 188: empressmediainc. Email: dave@empressmedia.com
Company 752: empressmediainc. Email: dmiller@empressmam.com
----------------------------------------------------------

Once again, its not clear what is a typo or not. For example, for the third pair the domains "avac.com" and "avac.org" may both be valid, or maybe one is a mistake. To be on the conservative side, we will say that only the following are probably typos: 

1) For the 4th pair, belomtmetals.com is likely a typo since the company name is "belmontmetals".
2) For the 6th pair, iptnamedesign.com is likely a typo since the company name is "iptnamedesignsinc".
3) For the 8th pair, pintoandtegerelectrica@.com also looks like a typo since the company name is "pintotegerelectric".
4) For the 10th pair, handytool.com and ehandytool.com differ by one letter, "e", which is also the last letter of "rochelle". This may be a typo.
5) For the 11th pair, the domain "allisoneden.comp" is likely incorrect and it should end with just ".com".

To wrap if, if we just want to count the unique domain names, without looking for typos the final answer is: there are 601 pairs of unique domain names.

If we want to count unique domain names, and say that two domain names are the same if they differ by a likely typo, the answer is: there are 601-5 = 596 pairs of unique domain names.

My preference is to say there are 596 pairs of unique domain names.

## Question 4

Considering only NTAs with at least 5 listed businesses, what is the average total savings and the total jobs created for each NTA?

To answer this, we will group the dataframe by the NTA column and filter the grouped dataframe to only retain NTAs with at least 5 businesses.
We take the resulting dataframe, group again by the NTA, then index into the "Total Savings" and "Job created" columns of the resulting dataframe, and finally compute their mean. This is accomplished in the code below. We save the resulting dataframe to the file "NTA_total_savings_jobs_created.csv".

In [14]:
grouped = df.groupby('Neighborhood Tabulation Area (NTA) (2020)')
grouped = grouped.filter(lambda x:x['Company Name'].nunique()>=5)

result = grouped.groupby('Neighborhood Tabulation Area (NTA) (2020)')[['Total Savings','Job created']].mean()

result.to_csv("NTA_total_savings_jobs_created.csv")
result

,Total Savings,Job created
Neighborhood Tabulation Area (NTA) (2020),,
BK0101,10367.961795,5.333333
BK0104,21158.253077,10.388889
BK0202,74011.255897,NaN
BK0261,12876.958500,NaN
BK0301,57934.141667,4.000000
BK0502,13810.223333,NaN
BK0503,62669.078889,59.285714
BK0601,25358.179524,10.777778
BK0702,17596.188571,7.666667
